In [1]:
#import libaries
import pandas as pd
import numpy as np
import matplotlib.pyplot as  plt
import seaborn as sns
from datetime import datetime
import datetime

In [2]:
col_list=["VisitDateTime","ProductID","UserID","Activity","OS"]

In [3]:
vis=pd.read_csv("VisitorLogsData.csv",usecols=col_list)

In [4]:
vis=vis.dropna(subset=["UserID"])

In [5]:
vis["Activity"]=vis["Activity"].str.lower()

In [6]:
vis.Activity.fillna(value="pageload",inplace=True)#if he has not clicked it can be assumed to be browsing

In [7]:
vis["ProductID"]=vis["ProductID"].str.title()#after seeing values we found out disperancy in casig converting to camel case

In [8]:
vis.ProductID.fillna(value="Product101",inplace=True)#giving in question

In [9]:
vis["OS"]=vis["OS"].str.lower()

In [10]:
most_frequent_os=vis.OS.max()

In [11]:
vis['OS']=vis.OS.mask(vis['OS'].map(vis['OS'].value_counts(normalize=True)) < 0.05, 'rare')#less than 5%


In [12]:
vis.VisitDateTime.fillna("2018-05-07 00:00:01",inplace=True)#notdroping as time is the most imp feature and we can't oblige it

In [13]:
vis["VisitDateTime"]=vis["VisitDateTime"].astype("category")
vis["VisitDateTime"]=vis["VisitDateTime"].astype('datetime64[ns]')

In [14]:
vis=vis[vis["VisitDateTime"]>"1994-01-01"]
vis=vis[vis["VisitDateTime"]<"2018-05-28"]

In [15]:
vis["Date"]=pd.to_datetime(vis["VisitDateTime"]).dt.date

In [16]:
vis["Date"]=vis["Date"].astype("category")
vis["Date"]=vis["Date"].astype('datetime64[ns]')

In [17]:
last_day=datetime.datetime(2018,5,28)

In [18]:
vis["vintage"]=-(vis["Date"]-last_day)

In [19]:
vis["vintage"]=vis["vintage"].dt.days

In [20]:
user_vintage=vis.groupby(["UserID"])["vintage"].max()

In [21]:
user_vintage

UserID
U100002      10
U100003      15
U100004      21
U100005      21
U100006       7
           ... 
U136959      21
U136961      21
U136963    2902
U136964      21
U136965      21
Name: vintage, Length: 33457, dtype: int64

In [22]:
sample_submission=pd.read_csv("sample_submission_M7Vpb9f.csv")

In [23]:
feature= pd.merge(
    left=sample_submission,
    right=user_vintage,
    how="left",
    on='UserID',
    copy=True)

In [24]:
feature["User_Vintage"]=feature["vintage"]

In [25]:
feature.drop(columns="vintage",inplace=True)

In [26]:
feature.fillna(value=0,inplace=True)

In [27]:
vis=vis[vis['Date']>"2018-05-07"]

In [28]:
Most_Active_OS=vis.groupby(["UserID"])["OS"].max()

In [29]:
feature= pd.merge(
    left=feature,
    right=Most_Active_OS,
    how="left",
    on='UserID',
    copy=True)

In [30]:
feature["Most_Active_OS"]=feature["OS"]

In [31]:
feature.drop(columns=["OS"],inplace=True)

In [32]:
feature.Most_Active_OS.fillna(value=most_frequent_os,inplace=True)#filling na with most frequent os

In [33]:
Recently_Viewed_Product=vis.groupby(['UserID'])['ProductID'].max()

In [34]:
feature= pd.merge(
    left=feature,
    right=Recently_Viewed_Product,
    how="left",
    on='UserID',
    copy=True)

In [35]:
feature['Recently_Viewed_Product']=feature['ProductID']

In [36]:
feature.drop(columns="ProductID",inplace=True)

In [37]:
feature.Recently_Viewed_Product.fillna(value="Product101",inplace=True)

In [38]:
feature.head()

,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days,Clicks_last_7_days
0,U100002,1,1,10.0,P12345,android,Pr101097,1,1
1,U100003,1,1,15.0,P12345,windows,Product101,1,1
2,U100004,1,1,21.0,P12345,windows,Product101,1,1
3,U100005,1,1,21.0,P12345,android,Pr102072,1,1
4,U100006,1,1,7.0,P12345,android,Pr101111,1,1


In [39]:
usr=pd.read_csv("userTable.csv")

In [40]:
usr["Signup Date"]=usr["Signup Date"].astype("datetime64[ns]")

In [41]:
combined_data=pd.merge(left=vis,right=usr,how="left",on="UserID",copy=True)

In [42]:
#combined_data.info()

In [43]:
combined_data=combined_data[combined_data["Date"]>"2018-05-13"]

In [44]:
No_Of_Products_Viewed_15_Days=combined_data.groupby(["UserID"])["ProductID"].count()

In [45]:
feature= pd.merge(
    left=feature,
    right=No_Of_Products_Viewed_15_Days,
    how="left",
    on='UserID',
    copy=True)

In [46]:
feature["No_Of_Products_Viewed_15_Days"]=feature["ProductID"]

In [47]:
feature.drop(columns="ProductID",inplace=True)

In [48]:
feature.No_Of_Products_Viewed_15_Days.fillna(value=0,inplace=True)#not viwed any product in last 15 days

In [49]:
combined_data_pageload=combined_data[combined_data["Activity"]=="pageload"]

In [50]:
Most_Viewed_product_15_Days=combined_data_pageload.groupby(["UserID"])["ProductID"].max()

In [51]:
feature= pd.merge(
    left=feature,
    right=Most_Viewed_product_15_Days,
    how="left",
    on='UserID',
    copy=True)

In [52]:
feature["Most_Viewed_product_15_Days"]=feature["ProductID"]

In [53]:
feature.drop(columns="ProductID",inplace=True)

In [54]:
feature.Most_Viewed_product_15_Days.fillna(value="Product101",inplace=True)#given in question

In [55]:
combined_data=combined_data[combined_data["Date"]>"2018-05-21"]

In [57]:
No_of_days_Visited_7_Days=combined_data.groupby(["UserID","Date"])["Date"].count()

In [58]:
feature= pd.merge(
    left=feature,
    right=No_of_days_Visited_7_Days,
    how="left",
    on='UserID',
    copy=True)

In [61]:
feature["No_of_days_Visited_7_Days"]=feature["Date"]

In [63]:
feature.drop(columns=["Date"],inplace=True)

In [64]:
feature.No_of_days_Visited_7_Days.fillna(value=0,inplace=True)

In [65]:
combined_data_pageload=combined_data_pageload[combined_data_pageload["Date"]>"2018-05-20"]

In [66]:
Pageloads_last_7_days=combined_data_pageload.groupby(["UserID"])["Activity"].count()

In [67]:
feature= pd.merge(
    left=feature,
    right=Pageloads_last_7_days,
    how="left",
    on='UserID',
    copy=True)

In [68]:
feature["Pageloads_last_7_days"]=feature["Activity"]

In [69]:
feature.drop(columns="Activity",inplace=True)

In [70]:
feature.Pageloads_last_7_days.fillna(value=0,inplace=True)

In [71]:
combined_data=combined_data[combined_data["Activity"]=="click"]

In [72]:
Clicks_last_7_days=combined_data.groupby(["UserID"])["Activity"].count()

In [73]:
feature= pd.merge(
    left=feature,
    right=Clicks_last_7_days,
    how="left",
    on='UserID',
    copy=True)

In [74]:
feature["Clicks_last_7_days"]=feature["Activity"]

In [75]:
feature.drop(columns="Activity",inplace=True)

In [76]:
feature.Clicks_last_7_days.fillna(value=0,inplace=True)#not active

In [77]:
feature.head()

,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days,Clicks_last_7_days
0,U100002,0.0,4.0,10.0,Pr101097,android,Pr101097,0.0,0.0
1,U100003,3.0,3.0,15.0,Pr100079,windows,Product101,1.0,2.0
2,U100004,0.0,20.0,21.0,Product101,windows,Product101,1.0,0.0
3,U100005,2.0,4.0,21.0,Pr102072,android,Pr102072,2.0,0.0
4,U100006,0.0,1.0,7.0,Pr101111,android,Pr101111,1.0,0.0


In [78]:
feature=feature.astype({'No_of_days_Visited_7_Days':'int','Pageloads_last_7_days':'int','Clicks_last_7_days':'int','No_Of_Products_Viewed_15_Days':'int','User_Vintage':'int'})

In [79]:
feature.to_csv("final.csv",index=False)